In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/mnt/c/Users/Dleg/Data/MOPC/MOPC/Enriquecimiento de Datos/Data Cleaning/PyP/Datasets/Datasets enviados/Cartera_sin_enriquecer - Estados_idcliente - 2602-4.csv')

In [3]:
df.head()

,IDMOROSO,IDCLIENTE,ESTADO,NOMBRE
0,20185650929,AMEX_TNRM,SKIP LEVEL1 OK,ALEJANDRO LUCANO
1,20217233047,AMEX_PARM,NO UBICADO,RAUL ANIBAL MELGAREJO
2,20249174182,AMEX_PBRM,INTERESADO,JUAN BECERRA
3,23129870869,AMEX_TNLM,SKIP LEVEL1 OK,JUAN ANTONIO GUERRA
4,27052550322,AMEX_TNRM,SKIP LEVEL1 OK,MIRTA MABEL NAVE


In [4]:
df.shape

(3242, 4)

In [5]:
df['IDMOROSO'] = df['IDMOROSO'].str.strip()

In [6]:
df['NRO_DOC'] = df['IDMOROSO']

In [7]:
df = df.dropna(subset=['NRO_DOC'])

In [8]:
df.shape

(3242, 5)

## 12.2) DNI Cleaning

Limpiamos el dataframe __df__, ya que dentro de DATO_A_ENVIAR tenemos 9 tipos de casos diferentes de datos:

- DNI 12345678
- DNI12345678
- DNIE1234678
- LC  1234567
- LE  1234567
- CUIT00123456780
- 00123456780 --> Es un CUIT.
- 00 12345678 0 --> Es un CUIT.
- DNI 00123456780 --> Dice DNI pero es un CUIT de una persona __jurídica__ (TO DO)

Para eso definimos 5 funciones:

- __Tipo_DOC__: Se definen dos funciones para determinar el tipo de DNI. Una inicial, y otra para corregir sobre el dataframe resultante.
- __Standard DNI__: Estandariza los DNI para 7 de los casos. Esta función debe ejecutarse en primer lugar.
- __DNI Spaces__: Elimina los espacios.
- __DNI Zero__: Elimina los ceros iniciales de ciertos CUITs para que coincidan con los DNI.
- __Cuit__: Decisión sobre dejar un cuit o volverlo DNI

In [9]:
def tipo_doc_1 (string_dni):
    
    if 'DNI' in string_dni:
        tipo = 'DNI'
    
    elif 'LC' in string_dni:
        tipo = 'LC'
    
    elif 'LE' in string_dni:
        tipo = 'LE'
    
    elif 'DNIE' in string_dni:
        tipo = 'DNIE'    
    
    elif ((string_dni.startswith(('CUIT30','CUIT33','CUIT34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'
        
    elif ((string_dni.startswith(('30','33','34'))) & (len(string_dni)>8)):
        tipo = 'CUIT'

    elif ((string_dni.startswith(('20','23','24','27'))) & (len(string_dni)>8)):
        tipo = 'DNI'

    elif ((string_dni.startswith(('CUIT20','CUIT23','CUIT24','CUIT27'))) & (len(string_dni)>8)):
        tipo = 'DNI'
        
    else:
        tipo = 'DNI'
    
    return tipo

In [10]:
def tipo_doc_2 (string_dni, tipo_original):
    
    if (len(string_dni) > 8) & (tipo_original == 'DNI'):
        tipo = 'CUIT'
    else:
        tipo = tipo_original
    return tipo

In [11]:
def standard_dni (string_dni):
    """DNI separation by each case.
    """
    
    if 'DNI ' in string_dni:
        dni = string_dni.split('DNI ')[1]
        
    elif 'DNIE' in string_dni:
        dni = string_dni.split('DNIE')[1]
    
    elif 'DNI' in string_dni:
        dni = string_dni.split('DNI')[1]
    
    elif 'LC  ' in string_dni:
        dni = string_dni.split('LC  ')[1]
    
    elif 'LE  ' in string_dni:
        dni = string_dni.split('LE  ')[1]
    
    elif 'CUIT' in string_dni:
        dni = string_dni.split('CUIT')[1]
        
    else:
        dni = string_dni
        
    return dni

In [12]:
def dni_characters (string_dni):
    
    if ' ' in string_dni:
        dni = string_dni.replace(' ','')
        
    elif '-' in string_dni:
        dni = string_dni.replace('-','')

    else:
        dni = string_dni
    
    return dni

In [13]:
def cuit (string_dni):
    
    if ((string_dni.startswith('20')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('23')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]
        
    elif ((string_dni.startswith('24')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    elif ((string_dni.startswith('27')) & (len(string_dni)>8)):
        cuit = string_dni [2:-1]

    else:
        cuit = string_dni
        
    return cuit

In [14]:
def dni_zero (string_dni):
    
    if string_dni.startswith('0'):
        dni = string_dni [1:]

    else:
        dni = string_dni
    
    return dni

En primer lugar, trimmeamos los espacios que sobran:

In [15]:
df['NRO_DOC'] = [(str(df['NRO_DOC'][i]).strip()) for i in range (df['NRO_DOC'].count())]

Se implementan estas funciones:

Para determinar el __TIPO DE DOCUMENTO__ en primer lugar:

In [16]:
df['TIPO_DOC'] = [tipo_doc_1(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

Para la limpieza de DNIs:

In [17]:
df['NRO_DOC'] = [standard_dni(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [18]:
df['NRO_DOC'] = [dni_characters(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [19]:
df['NRO_DOC'] = [cuit(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [20]:
df['NRO_DOC'] = [dni_zero(df['NRO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

Para corregir el __TIPO DE DOCUMENTO__ sobre el dataframe resultante

In [21]:
df['TIPO_DOC'] = [tipo_doc_2(df['NRO_DOC'][i],df['TIPO_DOC'][i]) for i in range (df['NRO_DOC'].count())]

In [22]:
df.head()

,IDMOROSO,IDCLIENTE,ESTADO,NOMBRE,NRO_DOC,TIPO_DOC
0,20185650929,AMEX_TNRM,SKIP LEVEL1 OK,ALEJANDRO LUCANO,18565092,DNI
1,20217233047,AMEX_PARM,NO UBICADO,RAUL ANIBAL MELGAREJO,21723304,DNI
2,20249174182,AMEX_PBRM,INTERESADO,JUAN BECERRA,24917418,DNI
3,23129870869,AMEX_TNLM,SKIP LEVEL1 OK,JUAN ANTONIO GUERRA,12987086,DNI
4,27052550322,AMEX_TNRM,SKIP LEVEL1 OK,MIRTA MABEL NAVE,5255032,DNI


In [23]:
df.tail()

,IDMOROSO,IDCLIENTE,ESTADO,NOMBRE,NRO_DOC,TIPO_DOC
3237,DNIE95671863,BSR_TARDIA,NO VISTO,"EDGAR ALFONSO,CASTELLANO PI#A",95671863,DNI
3238,DNIE95682881,BSR_TARDIA,NO VISTO,"DIEGO RAMON,CABRERA DOMINGUEZ",95682881,DNI
3239,DNIE95789827,BSR_TARDIA,NO VISTO,"TARSO AUGUSTO,BARROS CAMPOS",95789827,DNI
3240,DNIE95881382,BSR_TARDIA,NO VISTO,"TEQUEILA,BATISTA SILVA TAVARE",95881382,DNI
3241,DNIE95908456,BSR_TARDIA,NO VISTO,"RICHARD,CARDOZO VAZ TRAJANO",95908456,DNI


In [24]:
df_export = df.rename(columns={'NRO_DOC':'DOCUMENTO','TIPO_DOC':'TIPO'})

In [25]:
df_export = df_export [['TIPO','DOCUMENTO','NOMBRE']]

In [26]:
df_export.head()

,TIPO,DOCUMENTO,NOMBRE
0,DNI,18565092,ALEJANDRO LUCANO
1,DNI,21723304,RAUL ANIBAL MELGAREJO
2,DNI,24917418,JUAN BECERRA
3,DNI,12987086,JUAN ANTONIO GUERRA
4,DNI,5255032,MIRTA MABEL NAVE


In [27]:
df_export.shape

(3242, 3)

In [28]:
# Export to csv cuando se requiera enviar